In [55]:
from tensorflow.keras.models import model_from_json
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [56]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def weighted_absolute_percentage_error(y_true, y_pred):
    return 100 / len(y_true) * np.sum(np.abs((y_true - y_pred) / y_true))

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    return 100 / sum(y_true) * np.sum(np.abs(y_true - y_pred))

In [73]:
all_values = pd.read_csv(Path("Resources/all_values.csv"))
all_values['date'] = pd.to_datetime(all_values['date'], format='%Y-%m')
all_values.set_index('date', inplace=True)
all_values.drop(columns=['Clothing and footwear'], inplace=True)
#all_values.dtypes

In [74]:
scaler_all = StandardScaler()
all_scaler = scaler_all.fit(all_values)
all_values_scaled = all_scaler.transform(all_values)

all_values_scaled_df = pd.DataFrame(all_values_scaled, columns=all_values.columns, index=all_values.index)
all_values_scaled_df.head()

,Composite,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,
2008-06-01,-1.120010,-0.664592,-0.452623,-0.981467,-0.848043,2.568481,-1.329134,-1.744835,-1.183779,-1.929083,-0.678934,-1.897463,-1.378353,-1.491493,-1.190403,-1.408245
2008-07-01,-1.126250,-0.637850,-0.535303,-1.096815,-0.935423,2.568481,-1.280302,-1.689366,-1.099725,-1.901250,-0.678934,-1.884047,-1.394462,-1.497273,-1.158831,-1.397238
2008-08-01,-1.131451,-0.605399,-0.435669,-0.975607,-0.862901,2.568481,-1.280302,-1.633897,-1.057698,-1.901250,-0.795051,-1.763302,-1.346137,-1.503054,-1.179879,-1.375226
2008-09-01,-1.136131,-0.720345,-0.478985,-0.859073,-0.752012,2.568481,-1.270535,-1.550693,-1.116536,-1.776002,-0.802309,-1.776718,-1.362245,-1.474150,-1.179879,-1.375226
2008-10-01,-1.152772,-0.881425,-1.320192,-1.192878,-1.010260,1.604616,-1.348667,-1.516025,-1.108130,-1.803835,-1.092603,-1.763302,-1.394462,-1.474150,-1.295643,-1.397238


In [75]:
# load json and create model
file_path = ("Resources/model.json")
with open("Resources/model.json", "r") as json_file:
    model_json = json_file.read()
model = model_from_json(model_json)

# load weights into new model
file_path = "Resources/model.h5"
model.load_weights(file_path)

In [76]:
y = all_values_scaled_df["Composite"]

In [77]:
X = all_values_scaled_df.drop(columns = "Composite")

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [79]:
# Generate predictions
y_pred = model.predict(X_test)

# Calculate R-Squared
r2 = r2_score(y_test, y_pred)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Absolute Percentage Error
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

# Calculate Normalized Root Mean Squared Error
nrmse = rmse / (np.max(y_test) - np.min(y_test))

# Calculate Weighted Absolute Percentage Error
wape = weighted_absolute_percentage_error(y_test, np.squeeze(y_pred))

# Calculate Weighted Mean Absolute Percentage Error
wmape = weighted_mean_absolute_percentage_error(y_test, np.squeeze(y_pred))

2/2 [==============================] - 0s 2ms/step


In [80]:
print(f"R-Squared: {r2}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Normalized Root Mean Squared Error: {nrmse}")
print(f"Weighted Absolute Percentage Error: {wape}")
print(f"Weighted Mean Absolute Percentage Error: {wmape}")

R-Squared: 0.9988016716169702
Mean Absolute Error: 0.022273150665966152
Mean Absolute Percentage Error: 168.8911436346315
Mean Squared Error: 0.0010756084224295243
Root Mean Squared Error: 0.03279646966411971
Normalized Root Mean Squared Error: 0.008837644892259614
Weighted Absolute Percentage Error: 4.226816274760644
Weighted Mean Absolute Percentage Error: -38.01716290166081
